In [1]:
# NEXT - add new stats to game score model

In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
con = sqlite3.Connection('db.db')

In [4]:
cursor = con.cursor()

In [37]:
df = pd.read_sql_query('''select *, 
(
(pts * 0.04) + 
case when closeqf = 1 then ptsqf * 0.04 else ptsqf * 0.02 end + 
yds*0.0008 + 
(tds * 0.15) + 
(tos * 0.08) + 
(close * 0.1) + 
(closeqf * 0.2) +
(sacks * 0.015) + 
(sackyds * 0.011) + 
(rtns * 0.3) + 
(ot * 0.15)
) as score from summary_view;''', con=con)

In [33]:
df.describe()

,week,season,pts,ptsqf,yds,tds,tos,fdpc,sacks,sackyds,rtns,drives,drvtdpc,close,closeqf,ot,score
count,803.000000,803.000000,803.000000,803.000000,803.000000,803.000000,803.000000,738.000000,803.000000,803.000000,803.000000,803.000000,803.0,803.000000,803.000000,803.000000,803.000000
mean,9.428394,2019.002491,47.254047,13.287671,706.225405,5.122042,2.673724,0.250678,4.792030,32.028643,0.049813,22.178082,0.0,0.764633,0.738481,0.047323,4.570160
std,5.235196,0.817510,14.233630,7.764340,122.752484,2.156600,1.616049,0.433697,2.349794,17.376868,0.228863,3.216140,0.0,0.424493,0.439736,0.212460,1.126895
min,1.000000,2018.000000,6.000000,0.000000,393.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.000000,0.0,0.000000,0.000000,0.000000,1.736600
25%,5.000000,2018.000000,38.000000,7.000000,616.500000,4.000000,1.500000,0.000000,3.000000,19.000000,0.000000,20.000000,0.0,1.000000,0.000000,0.000000,3.743600
50%,9.000000,2019.000000,47.000000,13.000000,701.000000,5.000000,3.000000,0.000000,5.000000,30.000000,0.000000,22.000000,0.0,1.000000,1.000000,0.000000,4.500600
75%,14.000000,2020.000000,57.000000,17.000000,787.000000,7.000000,4.000000,0.750000,6.000000,44.000000,0.000000,24.000000,0.0,1.000000,1.000000,0.000000,5.314000
max,18.000000,2020.000000,105.000000,44.000000,1091.000000,12.000000,8.000000,1.000000,12.000000,98.000000,2.000000,38.000000,0.0,1.000000,1.000000,1.000000,9.734800


In [34]:
df = df.drop(['date', 'time'], axis=1).sort_values('score', ascending=False)  

In [35]:
df[(df['week'] == 4) & (df['season'] == 2020)]

,match_id,week,season,teama_name,teamh_name,pts,ptsqf,yds,tds,tos,fdpc,sacks,sackyds,rtns,drives,drvtdpc,close,closeqf,ot,score
586,202010040dal,4,2020,Cleveland Browns,Dallas Cowboys,87,32,1074,11,3,0.0,5,22,0,25,0,0,0,0,7.1862
589,202010040tam,4,2020,Los Angeles Chargers,Tampa Bay Buccaneers,69,10,808,8,3,1.0,2,12,0,23,0,1,1,0,5.7084
582,202010010nyj,4,2020,Denver Broncos,New York Jets,65,25,680,5,3,0.0,6,54,0,24,0,0,0,0,5.3180
587,202010040det,4,2020,New Orleans Saints,Detroit Lions,64,8,673,9,2,1.0,5,33,0,18,0,1,0,0,5.3064
591,202010040mia,4,2020,Seattle Seahawks,Miami Dolphins,54,25,856,5,3,0.0,3,20,0,19,0,0,1,0,5.2998
594,202010040sfo,4,2020,Philadelphia Eagles,San Francisco 49ers,45,20,684,5,4,0.0,8,56,0,22,0,1,1,0,5.2532
593,202010040rai,4,2020,Buffalo Bills,Las Vegas Raiders,53,20,720,6,2,0.0,3,28,0,19,0,1,1,0,5.2090
590,202010040cin,4,2020,Jacksonville Jaguars,Cincinnati Bengals,58,18,934,5,2,1.0,4,11,0,21,0,1,1,0,5.1782
596,202010050gnb,4,2020,Atlanta Falcons,Green Bay Packers,46,10,730,6,0,0.0,5,48,0,18,0,1,1,0,4.6270
583,202010040car,4,2020,Arizona Cardinals,Carolina Panthers,52,10,706,7,2,1.0,1,0,0,19,0,1,1,0,4.5698


In [36]:
df[df['season'] == 2020].head(30)

,match_id,week,season,teama_name,teamh_name,pts,ptsqf,yds,tds,tos,fdpc,sacks,sackyds,rtns,drives,drvtdpc,close,closeqf,ot,score
777,202101030buf,17,2020,Miami Dolphins,Buffalo Bills,82,34,909,9,6,0.0,2,20,1,29,0,1,1,0,8.0472
658,202011080buf,9,2020,Seattle Seahawks,Buffalo Bills,78,31,839,9,4,0.0,12,57,0,23,0,1,1,0,7.8082
614,202010180oti,6,2020,Houston Texans,Tennessee Titans,78,28,1019,11,2,1.0,4,37,0,21,0,1,1,1,7.7822
758,202012250nor,16,2020,Minnesota Vikings,New Orleans Saints,85,27,947,12,2,0.0,2,17,0,22,0,1,1,0,7.7146
741,202012140cle,14,2020,Baltimore Ravens,Cleveland Browns,89,35,878,12,1,1.0,4,22,0,23,0,1,0,0,7.2444
632,202010250cin,7,2020,Cleveland Browns,Cincinnati Bengals,71,34,866,9,3,1.0,4,35,0,18,0,1,1,0,7.2278
586,202010040dal,4,2020,Cleveland Browns,Dallas Cowboys,87,32,1074,11,3,0.0,5,22,0,25,0,0,0,0,7.1862
746,202012200dal,15,2020,San Francisco 49ers,Dallas Cowboys,74,26,749,8,4,1.0,4,28,1,27,0,1,1,0,7.0872
788,202101030htx,17,2020,Tennessee Titans,Houston Texans,79,20,949,9,2,0.0,6,36,0,21,0,1,1,0,7.0152
557,202009200dal,2,2020,Atlanta Falcons,Dallas Cowboys,79,26,950,9,3,0.0,2,11,0,25,0,1,1,0,7.0010
